In [1]:
# 安装用到的库
import json
import numpy as np

In [2]:
# 这个类的目的是把json文件读入成字典格式，该类把json文件中的size，A，D和M分别
# 设置为类的四个属性。
class InputData:
    def __init__(self, path):
        self.__path = path
        self.DataLoad()

    def DataLoad(self):
        with open(self.__path, "r") as inputFile:
            inputData = json.load(inputFile)

        self.Size = inputData["Size"]

        # 使用数组是为了更加直观np.array()
        self.Transport_Matrix = np.array(inputData['A']) # 运输数量矩阵，从设备 i 到设备 j
        self.Distance_Matrix = np.array(inputData['D'])  # Station之间的距离矩阵，从k到l
        self.Mind_Entfernung = np.array(inputData['M'])  # 设备Anlage i和j之间，至少应保持的距离

    def __str__(self):
        # str函数用于直接打印inputData对象时，输出这个json文件涉及多少个设备的排列问题
        return f'Quadratic Assignment Problem von {self.Size} Anlagen. '




In [3]:
data = InputData('scr12.json')
print(data.Size)
print(data)
print(data.Distance_Matrix)
print(type(data.Distance_Matrix))
print(data.Distance_Matrix[0][1])

12
Quadratic Assignment Problem von 12 Anlagen. 
[[0 1 2 3 1 2 3 4 2 3 4 5]
 [1 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 1 2]
 [4 3 2 3 3 2 1 2 2 1 0 1]
 [5 4 3 2 4 3 2 1 3 2 1 0]]
<class 'numpy.ndarray'>
1


In [4]:
data_15 = InputData("scr15.json")
data_20 = InputData("scr20.json")
data_25 = InputData("tai25b.json")

In [5]:
## 验证对称性
def checkSymA(data):
    result = 0
    for i in range(data.Size):
        for j in range(data.Size):
            if data.Transport_Matrix[i][j] != data.Transport_Matrix[j][i]:
                result += 1
                
            else:
                result += 0

    return result

def checkSymD(data):
    result = 0
    for i in range(data.Size):
        for j in range(data.Size):
            if data.Distance_Matrix[i][j] != data.Distance_Matrix[j][i]:
                result += 1
                
            else:
                result += 0

    return result

def checkSymM(data):
    result = 0
    for i in range(data.Size):
        # 因为矩阵的行数等于列数，也就是size
        for j in range(data.Size):
            if data.Mind_Entfernung[i][j] != data.Mind_Entfernung[j][i]:
                result += 1
                
            else:
                result += 0

    return result

In [6]:
# 经过下面三个单元格的输出可知，4个文件中的A矩阵都是对称的，
# 25台设备的情况下，D矩阵和M矩阵不对称，D不对称是由单行线导致，M不对称可能是不同机器的对最近距离的要求不同。
# scr12，15和20的情况下，三个矩阵都是对称的，解决问题时只需要考虑是否要将最近距离M考虑在内，可以用子类和父类来做
# 父类是不考虑机器之间应保持的最近距离，只用A和D矩阵来做，而子类需要重写父类的运算过程，加一个if，判断
# 对应位置的距离D和对应机器的最小距离M哪个更大，更大的那个用来做计算。这个计算过程不一定要一步一步算，可以先来
# 构建一个用于计算的距离矩阵（这个矩阵包含D中和M中更大的那个值），然后用这个矩阵和A矩阵做矩阵相乘, 然后用.sum()
print(checkSymA(data))
print(checkSymA(data_15))
print(checkSymA(data_20))
print(checkSymA(data_25))

0
0
0
0


In [7]:
print(checkSymD(data))
print(checkSymD(data_15))
print(checkSymD(data_20))
print(checkSymD(data_25))

0
0
0
498


In [8]:
print(checkSymM(data))
print(checkSymM(data_15))
print(checkSymM(data_20))
print(checkSymM(data_25))

0
0
0
36


In [9]:
# 矩阵或数组之间相乘，得到的结果是对应位置元素之间相乘的结果
a = np.array([[1,2], [3,2]])
b = np.array([[2,4], [1,3]])
print(a*b)
print((a*b).sum())
print(sum(a*b))
print(len(a))
print(len(a*b))

[[2 8]
 [3 6]]
19
[ 5 14]
2
2


In [10]:
print(list(range(data.Size)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [11]:
gesamt_12 = data.Distance_Matrix * data.Transport_Matrix
print(gesamt_12.sum())
print(sum(gesamt_12))

50116
[1180 8742 3208 9982 1755 2902 8129 2805 2790 4874  709 3040]


In [12]:
import copy
robot=[1,2,3]
see=[]
see=copy.deepcopy(robot)
print(see)

[1, 2, 3]


In [13]:
# 这个类是将输入的数据进行预处理，以输出gesamtTransportLeistung（分两种情况）作为属性。
# 类似讲义中的ProcessingTime
# 用inputData作为输入
import copy
class OutputData:
    def __init__(self, inputData, withMinimumDistance = False):
        self.InputData = inputData
        self.Size = self.InputData.Size
        self.A = self.InputData.Transport_Matrix
        self.D = self.InputData.Distance_Matrix
        self.M = self.InputData.Mind_Entfernung

        # InitialPermutation 是对Station的排列，可理解为把这个Station送给某个机器  -->重新排列后的D矩阵在Solution类中做！！！！
        self.InitialPermutation = list(range(self.InputData.Size)) # 这里输出的是初始排列，即fcfs。因此这里先不用考虑对Station的排列
        self.WithMinimumDistance = withMinimumDistance
        self.gesamtTransportLeistung = self.CalculateInitialTransport()
        
        

    def CalculateInitialTransport(self):
        if self.WithMinimumDistance == False:
            total = (self.A * self.D).sum()
        else:
            # 当考虑设备间的最短距离时，需要比较D矩阵和M矩阵对应元素的大小，留下大的。
            # 通过观察json中的D和M，M是个稀疏矩阵，因此我只需要创建一个D_copy，把M更大的元素赋给这个副本矩阵，，然后用副本矩阵跟A相乘
            DistanceWithMin = copy.deepcopy(self.D)

            # fcfs的情况下，0号设备放在0号Station，并以此类推，因此不用考虑对矩阵D的重新排列
            # Solution中需要先把D按照Permutation排列好，然后再计算
            for i in self.InitialPermutation:
                for j in self.InitialPermutation:
                    if self.D[i][j] < self.M[i][j]:
                        DistanceWithMin[i][j] = self.M[i][j]
            total = (self.A * DistanceWithMin).sum()
        return total


    def __str__(self):
        return f'OutputData: \nQuadratic Assignment Problem von {self.InputData.Size} Anlagen, mit einer initiallen Transportleistung von {self.gesamtTransportLeistung}.'
        
output_12 = OutputData(data)
print(len(output_12.D))
print(output_12)

output_12_m = OutputData(data, True)
print(output_12_m)

print(output_12.D)


12
OutputData: 
Quadratic Assignment Problem von 12 Anlagen, mit einer initiallen Transportleistung von 50116.
OutputData: 
Quadratic Assignment Problem von 12 Anlagen, mit einer initiallen Transportleistung von 50998.
[[0 1 2 3 1 2 3 4 2 3 4 5]
 [1 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 1 2]
 [4 3 2 3 3 2 1 2 2 1 0 1]
 [5 4 3 2 4 3 2 1 3 2 1 0]]


In [14]:
output_15 = OutputData(data_15)
output_20 = OutputData(data_20)

In [15]:
D_c = np.array([[0,2,2,3,1,2,3,4,2,3,4,5],
[2,0,1, 2, 2, 1, 2, 3, 3, 2, 3, 4], [2, 1, 0, 1, 3, 2, 1, 2, 4, 3, 2, 3], [3, 2, 1, 0, 4, 3, 2, 1, 5, 4, 3, 2], [1, 2, 3, 4, 0, 1, 2, 3, 1, 2, 3, 4],
        [2, 1, 2, 3, 1, 0, 1, 2, 2, 1, 2, 3],
        [3, 2, 1, 2, 2, 1, 0, 1, 3, 2, 1, 2],
        [4, 3, 2, 1, 3, 2, 1, 0, 4, 3, 2, 1],
        [2, 3, 4, 5, 1, 2, 3, 4, 0, 1, 2, 3],[3, 2, 3, 4, 2, 1, 2, 3, 1, 0, 2, 2],[4, 3, 2, 3, 3, 2, 1, 2, 2, 2, 0, 2],[5, 4, 3, 2, 4, 3, 2, 1, 3, 2, 2, 0]])


print((D_c * data.Transport_Matrix).sum())

50998


In [16]:
# 矩阵和二维数组：按照permutation重新排列行和列

Dist= np.array([[1,2,3], [4,5,6],[7,8,9]])
new_perm = [2,0,1]
print(Dist)
# new_perm = np.array(range(12)[::-1])
# print(new_perm)
dist_zeil = Dist[new_perm, :] # 先把Dist的行按照perm进行排列，
print(dist_zeil)

dist_zeil = dist_zeil[:, new_perm] # 再把列进行从新排列
print(dist_zeil)


[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[7 8 9]
 [1 2 3]
 [4 5 6]]
[[9 7 8]
 [3 1 2]
 [6 4 5]]


In [17]:
res = 0
for i in range(12):
    for j in range(12):
        if output_12.D[i][j] < output_12.M[i][j]:
            res += 1

print(res)


print(list(range(12)))

6
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [18]:
# Permutation 是指station的排序；anlage矩阵不排序，默认从1（0）到n（n-1）排序。设备的排序也就相当于把这些设备分别安排在相应的Station中
class Solution:

    def __init__(self, outputData, permutation):
        self.OutputData = outputData
        self.Permutation = permutation
        self.TransportMatrix = self.OutputData.A
        self.ArrangedDMatrix = self.ArrangeDistanceMatrix() # 按照Permutation调整D矩阵的行列，得到Anlagen之间的距离，然后再与M矩阵直接比较
        self.UpdatedDMatrix = self.UpdateDMatrix() # 

        self.TotalTransport = -1  # 需要用EvaluationLogic类来计算具体的值

    def __str__(self):
        return f'Die Permutation {self.Permutation} führt zu einer Transportleistung von {self.TotalTransport}'

    def setPermutation(self, permutation):
        self.Permutation = permutation

    # 按照permutation重新排列D矩阵的行和列
    def ArrangeDistanceMatrix(self):
        arrangedDMatrix = self.OutputData.D[self.Permutation, :]
        arrangedDMatrix = arrangedDMatrix[:, self.Permutation]

        return arrangedDMatrix

    # 通过比较当前D矩阵和最小距离M矩阵，把得出更新后的D矩阵
    def UpdateDMatrix(self):
        DistanceWithMin = copy.deepcopy(self.ArrangedDMatrix)
        # res = 0
        for i in range(self.OutputData.Size):
            for j in range(self.OutputData.Size):
                if self.ArrangedDMatrix[i][j] < self.OutputData.M[i][j]:
                    DistanceWithMin[i][j] = self.OutputData.M[i][j]
                    # res += 1
        # print(res)
        return DistanceWithMin
        

sol_1 = Solution(output_12, list(range(12))) 
print(sol_1.ArrangedDMatrix)
print(sol_1.UpdatedDMatrix)


# 得出结论，我自己随便找的一个排列，会导致fcfs中的6个不满足最近距离的元素变成4个






[[0 1 2 3 1 2 3 4 2 3 4 5]
 [1 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 1 2]
 [4 3 2 3 3 2 1 2 2 1 0 1]
 [5 4 3 2 4 3 2 1 3 2 1 0]]
[[0 2 2 3 1 2 3 4 2 3 4 5]
 [2 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 2 2]
 [4 3 2 3 3 2 1 2 2 2 0 2]
 [5 4 3 2 4 3 2 1 3 2 2 0]]


In [19]:
# 把第11台设备放在了第一个Station     [0,11,1,10,2,9,3,8,4,7,5,6]
sol_2 = Solution(output_12, [0,11,1,10,2,9,3,8,4,7,5,6])
print(sol_2.ArrangedDMatrix)
print(sol_2.UpdatedDMatrix)

[[0 5 1 4 2 3 3 2 1 4 2 3]
 [5 0 4 1 3 2 2 3 4 1 3 2]
 [1 4 0 3 1 2 2 3 2 3 1 2]
 [4 1 3 0 2 1 3 2 3 2 2 1]
 [2 3 1 2 0 3 1 4 3 2 2 1]
 [3 2 2 1 3 0 4 1 2 3 1 2]
 [3 2 2 3 1 4 0 5 4 1 3 2]
 [2 3 3 2 4 1 5 0 1 4 2 3]
 [1 4 2 3 3 2 4 1 0 3 1 2]
 [4 1 3 2 2 3 1 4 3 0 2 1]
 [2 3 1 2 2 1 3 2 1 2 0 1]
 [3 2 2 1 1 2 2 3 2 1 1 0]]
[[0 5 1 4 2 3 3 2 1 4 2 3]
 [5 0 4 1 3 2 2 3 4 1 3 2]
 [1 4 0 3 1 2 2 3 2 3 2 2]
 [4 1 3 0 2 1 3 2 3 2 2 1]
 [2 3 1 2 0 3 1 4 3 2 2 1]
 [3 2 2 1 3 0 4 1 2 3 1 2]
 [3 2 2 3 1 4 0 5 4 1 3 2]
 [2 3 3 2 4 1 5 0 1 4 2 3]
 [1 4 2 3 3 2 4 1 0 3 1 2]
 [4 1 3 2 2 3 1 4 3 0 2 1]
 [2 3 2 2 2 1 3 2 1 2 0 2]
 [3 2 2 1 1 2 2 3 2 1 2 0]]


In [20]:
sol_3 = Solution(output_12, [0,2,4,6,8,10,11,9,7,5,3,1])
print(sol_3.ArrangedDMatrix)
print(sol_3.UpdatedDMatrix)

[[0 2 1 3 2 4 5 3 4 2 3 1]
 [2 0 3 1 4 2 3 3 2 2 1 1]
 [1 3 0 2 1 3 4 2 3 1 4 2]
 [3 1 2 0 3 1 2 2 1 1 2 2]
 [2 4 1 3 0 2 3 1 4 2 5 3]
 [4 2 3 1 2 0 1 1 2 2 3 3]
 [5 3 4 2 3 1 0 2 1 3 2 4]
 [3 3 2 2 1 1 2 0 3 1 4 2]
 [4 2 3 1 4 2 1 3 0 2 1 3]
 [2 2 1 1 2 2 3 1 2 0 3 1]
 [3 1 4 2 5 3 2 4 1 3 0 2]
 [1 1 2 2 3 3 4 2 3 1 2 0]]
[[0 2 1 3 2 4 5 3 4 2 3 1]
 [2 0 3 1 4 2 3 3 2 2 1 1]
 [1 3 0 2 1 3 4 2 3 2 4 2]
 [3 1 2 0 3 1 2 2 1 1 2 2]
 [2 4 1 3 0 2 3 1 4 2 5 3]
 [4 2 3 1 2 0 1 1 2 2 3 3]
 [5 3 4 2 3 1 0 2 1 3 2 4]
 [3 3 2 2 1 1 2 0 3 1 4 2]
 [4 2 3 1 4 2 1 3 0 2 1 3]
 [2 2 2 1 2 2 3 1 2 0 3 1]
 [3 1 4 2 5 3 2 4 1 3 0 2]
 [1 1 2 2 3 3 4 2 3 1 2 0]]


In [21]:
class SolutionPool:
    def __init__(self):
        self.Solutions = []

    def AddSolution(self, newSolution):
        self.Solutions.append(newSolution)

    def GetLowestTransport(self):
        self.Solutions.sort(key = lambda solution: solution.TotalTransport) # 升序排列。选出位于首位的solution，就是总运输量最小的

        return self.Solutions[0]

In [22]:
class EvaluationLogic:
    def __init__(self, withMinimumDistance = False):
        # self.InputData = inputData
        self.TotalTransport = None
        self.WithMinimumDistance = withMinimumDistance

    def DefineTotalTransport(self, currentSolution):
        # 这个函数需要再solver中调用，并且能同时输出考虑MinimumDistance和不考虑的结果
        if self.WithMinimumDistance == False:
            totalTransport = (currentSolution.TransportMatrix * currentSolution.ArrangedDMatrix).sum()
        else:
            totalTransport = (currentSolution.TransportMatrix * currentSolution.UpdatedDMatrix).sum()
        currentSolution.TotalTransport = totalTransport

        return totalTransport


In [23]:
el = EvaluationLogic() # 不考虑机器之间最小距离的情况下
el2 = EvaluationLogic(True) # 考虑机器之间最小距离的情况

el.DefineTotalTransport(sol_1)
print(sol_1)

el2.DefineTotalTransport(sol_1)
print(sol_1)



el.DefineTotalTransport(sol_2)
print(sol_2)

el2.DefineTotalTransport(sol_2)
print(sol_2)

el.DefineTotalTransport(sol_3)
print(sol_3)
el2.DefineTotalTransport(sol_3)
print(sol_3)

Die Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] führt zu einer Transportleistung von 50116
Die Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] führt zu einer Transportleistung von 50998
Die Permutation [0, 11, 1, 10, 2, 9, 3, 8, 4, 7, 5, 6] führt zu einer Transportleistung von 63118
Die Permutation [0, 11, 1, 10, 2, 9, 3, 8, 4, 7, 5, 6] führt zu einer Transportleistung von 63568
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 45322
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 46102


In [24]:
print([*range(len(output_12.D))])
print(list(range(12)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [25]:
print(sol_2)
EvaluationLogic().DefineTotalTransport(sol_2) 
sol_2.setPermutation([0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1])
print(sol_2)
EvaluationLogic().DefineTotalTransport(sol_2)    ######Solution对象的Permutation改变了，但是Transport没有！！
print(sol_2)

Die Permutation [0, 11, 1, 10, 2, 9, 3, 8, 4, 7, 5, 6] führt zu einer Transportleistung von 63568
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 63118
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 63118


In [26]:
EvaluationLogic().DefineTotalTransport(sol_2) 
sol_2.setPermutation([0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1])
print(sol_2)
print(sol_2.Permutation)
sol_2.ArrangedDMatrix = sol_2.ArrangeDistanceMatrix()
EvaluationLogic().DefineTotalTransport(sol_2)    ######Solution对象的Permutation改变了，但是Transport没有！！

print(sol_2)  ## 在修改了Permutation之后，只有当ArrangedDMatrix之后，总运输量才会改变！！！！

Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 63118
[0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1]
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 45322


In [27]:
# 到这里需要用启发式算法生成一个startSolution，这个方案要尽可能接近最优解
# 我选择的方法是ROS，随机数，即设置一个随机数种子，然后生成多个随机排列，每个排列包含的元素个数等于Size
# 首先尝试使用ROS

# from EvaluationLogic import *
import numpy
class ConstructiveHeuristics:
    def __init__(self, evaluationLogic, solutionPool):  # EL类分两种情况，是否考虑最短距离M。在这个类中不用特定指出是哪种情况
        self.RandomSeed = 999
        self.RandomRetiris = 40000
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool



    def ROS(self, outputData, iterations, seed):
        numpy.random.seed(seed)
        tmpSolution = Solution(outputData, outputData.InitialPermutation)  # 使用默认的排序，即给出的文件中的D矩阵中的Station排列
        bestCmax = numpy.inf


        for i in range(iterations):
            tmpPermutation = numpy.random.permutation(len(outputData.D))  # 随机排列能正常生成，
            tmpSolution.setPermutation(tmpPermutation)
            # tmpSolution.Permutation = tmpPermutation


            # 修改Solution类的属性：（因为D矩阵的排列方式取决于我们随机生成的Permutation）
            tmpSolution.ArrangedDMatrix = tmpSolution.ArrangeDistanceMatrix()
            # self.EvaluationLogic = EvaluationLogic()
            self.EvaluationLogic.DefineTotalTransport(tmpSolution)  # 这一行运行错误, 已解决！！

            # tmpSolution.TotalTransport = (tmpSolution.TransportMatrix * tmpSolution.ArrangedDMatrix).sum()

            if (tmpSolution.TotalTransport < bestCmax):
                bestCmax = tmpSolution.TotalTransport
                bestPerm = tmpPermutation
            

        bestSol = Solution(outputData, bestPerm)
        self.EvaluationLogic.DefineTotalTransport(bestSol)

        return bestSol

    def Run(self, outPutData):
        print('Generating an initial solution according to ROS. ')
        solution = None

        solution = self.ROS(outPutData, self.RandomRetiris, self.RandomSeed)
        
        self.SolutionPool.AddSolution(solution)


# 这个单元格应该已经不能运行了，6月5日我在这里调试成功ROS的运行，
# 但是没有把ConstructiveHeuristic做成一个类，也没有分类讨论是否需要考虑考虑M
# 6日我在下面一个单元格进行了更新，在EvaluationLogic类的构造器中加入了WithMinimumDistance参数，
# 这样这个参数会在solver中被传递给ConstructiveHeuristic
print(output_12.InitialPermutation)
print(output_12.D)
print(len(output_12.D))

s = Solution(output_12, output_12.InitialPermutation)
EvaluationLogic().DefineTotalTransport(s)
print(s)

# print(ROS(output_12, 1900, 999))
startSol = 


# 到这里为止，初始解决方案已经找到了，接下来开始进行优化Die Permutation [10  6  9  7  5  1  2  3 11  8  0  4] führt zu einer Transportleistung von 35540

In [28]:
# 到这里为止，初始解决方案已经找到了，接下来开始进行优化
# Die Permutation [10  6  9  7  5  1  2  3 11  8  0  4] führt zu einer Transportleistung von 35540
construct_no_m = ConstructiveHeuristics(EvaluationLogic(False), SolutionPool())
construct_with_m = ConstructiveHeuristics(EvaluationLogic(True), SolutionPool())

construct_no_m.Run(output_12)
print(construct_no_m.SolutionPool.GetLowestTransport())

construct_with_m.Run(output_12)
print(construct_with_m.SolutionPool.GetLowestTransport())



# Generating an initial solution according to ROS. 
# Die Permutation [10  6  9  7  5  1  2  3 11  8  0  4] führt zu einer Transportleistung von 35540


Generating an initial solution according to ROS. 
Die Permutation [ 2  1  9  0 10  4  5  6  8  7  3 11] führt zu einer Transportleistung von 35280
Generating an initial solution according to ROS. 
Die Permutation [ 4  2 10 11  6  7  3  9  8  1  5  0] führt zu einer Transportleistung von 60800


In [29]:
import numpy as np
np.random.seed(2022)
x = 0
while x < 10:
    perm = np.random.permutation(5)
    print(perm)
    x+=1

print(type(perm))

[2 3 0 1 4]
[4 2 3 0 1]
[3 2 1 4 0]
[4 0 2 3 1]
[2 1 4 3 0]
[3 4 1 2 0]
[0 1 3 4 2]
[3 4 1 0 2]
[0 2 4 3 1]
[1 3 2 0 4]
<class 'numpy.ndarray'>


In [30]:
# 创建一个Solver类，有两个目的，
# 第一，用solver对象把inputData读入，然后使用ConstructionPhase生成初始方案；
# 第二，使用优化算法，把初始方案逐步进行优化
class Solver:
    def __init__(self, inputData, seed, withMinimumDistance = False): # 默认不考虑M矩阵
        self.InputData = inputData
        self.Seed = seed
        self.RNG = numpy.random.default_rng(seed)
        self.OutputData = OutputData(self.InputData)

        self.EvaluationLogic = EvaluationLogic(withMinimumDistance)
        self.SolutionPool = SolutionPool()

        self.ConstructiveHeuristic = ConstructiveHeuristics(self.EvaluationLogic, self.SolutionPool)

    # 这里返回的是当前借助ROS算法得到的最优解，可以作为优化算法的
    def ConstructionPhase(self):
        self.ConstructiveHeuristic.Run(self.OutputData)
        bestInitialSol = self.SolutionPool.GetLowestTransport()
        
        print('Constructive Solution found. ')
        # print(bestInitialSol)

        return bestInitialSol






In [38]:
solver = Solver(data, 999, False)
print(solver.ConstructionPhase())



solver_m = Solver(data, 999, True)
print(solver_m.ConstructionPhase())

Generating an initial solution according to ROS. 
Constructive Solution found. 
Die Permutation [ 2  1  9  0 10  4  5  6  8  7  3 11] führt zu einer Transportleistung von 35280
Generating an initial solution according to ROS. 
Constructive Solution found. 
Die Permutation [ 4  2 10 11  6  7  3  9  8  1  5  0] führt zu einer Transportleistung von 60800


In [39]:
bestInitialSolution = solver.ConstructionPhase()
print(type(bestInitialSolution.Permutation))

print(list(bestInitialSolution.Permutation))

Generating an initial solution according to ROS. 
Constructive Solution found. 
<class 'numpy.ndarray'>
[2, 1, 9, 0, 10, 4, 5, 6, 8, 7, 3, 11]


 Scr12  12    31410 (OPT)   (8,6,3,2,10,1,5,9,4,7,12,11)

 Scr15  15    51140 (OPT)   (15,7,11,8,1,4,3,2,12,6,13,5,14,10,9)
 
 Scr20  20   110030 (OPT)   (20,7,12,6,4,8,3,2,14,11,18,9,19,15,16,17,13,5,10,1)

In [40]:
# 距离标准最优解还差很远
solver_15 = Solver(data_15, 999, False)
print(solver_15.ConstructionPhase())

solver_20 = Solver(data_20, 999, False)
print(solver_20.ConstructionPhase())

Generating an initial solution according to ROS. 
Constructive Solution found. 
Die Permutation [11  0 10  4  7  6  2  3  8 13  1 12  5  9 14] führt zu einer Transportleistung von 62612
Generating an initial solution according to ROS. 
Constructive Solution found. 
Die Permutation [11 17  4 13 19 12 18 15  1  3  0  8  9  2  7  5  6 16 10 14] führt zu einer Transportleistung von 155330


In [41]:
# 12 和15 的情况下，各自的初始解 ----> 急需进行局部搜索优化，现在ROS已经进行了40000次随机排列，只能优化到这个程度
print(output_15.CalculateInitialTransport())
print(output_20.CalculateInitialTransport())

77278
199318


In [35]:
# 现在进行ILS步骤，6月6日规划：
# 1. 先创建Neighborhood类，通过SwapMove 和TwoEdgeExchange两种方式，在初始方案（ROS）中进行变换，得到邻域解，
#    并评估全部邻域解！！ --> 注意继承关系
# 2. 然后创建ImprovementAlgorithm类，ILS算法就在这个类中。这里注意看Solver类中的RunLocalSearch, ImprovementPhase
class BaseNeighborhood():
    def __init__(self, inputData, initialPermutation, evaluationLogic, solutionPool):
        self.InputData = inputData
        self.OutputData = OutputData(inputData)
        self.Permutation = initialPermutation
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool

        self.Moves = []
        self.MoveSolutions = []

        self.Type = 'None'

    def DiscoverMoves(self):
        raise Exception('DiscoverMoves is implemented in the two child classes. ')

    # 这个函数只是用EL类计算当前moveSolution的总运输量，并返回这个moveSolution。会在EvaluateMovesBestImprovement函数中进行调用，不需要单独调用
    def EvaluateMove(self, move): 
        moveSolution = Solution(self.OutputData, move.Permutation)
        self.EvaluationLogic.DefineTotalTransport(moveSolution)
        return moveSolution

    # 这个函数只是把由DiscoverMoves函数生成的moves列表进行遍历，然后构造成Solution对象，再把Solution对象加入到MoveSolutions列表中。
    # 我们会在MakeBestMove函数中对moveSolution列表按照总运输量进行排序ASC，然后返回列表的第0个元素
    def EvaluateMovesBestImprovement(self):
        for move in self.Moves:
            moveSolution = self.EvaluateMove(move)
            self.MoveSolutions.append(moveSolution)

    def MakeBestMove(self):
        self.MoveSolutions.sort(key = lambda solution: solution.TotalTransport) # solution 的TotalTransport属性已经存在，因为EvaluateMove函数使用了EL
        bestNeighborhoodSolution = self.MoveSolutions[0]
        return bestNeighborhoodSolution

    # 用于修改initialPermutation，也就是说初始解可能会变。但是Beleg中估计不会用到
    def Update(self, permutation): 
        self.Permutation = permutation

        self.Moves.clear()
        self.MoveSolutions.clear()


    def LocalSearch(self, solution):
        hasSolutionImproved = True

        self.Update(solution.Permutation)
        self.DiscoverMoves()
        self.EvaluateMovesBestImprovement()

        bestNeighborhoodSolution = self.MakeBestMove()
        if bestNeighborhoodSolution.TotalTransport < solution.TotalTransport:
            print('New best Solution has been found! ')
            print(bestNeighborhoodSolution)

            self.SolutionPool.AddSolution(bestNeighborhoodSolution)
            solution.Permutation = bestNeighborhoodSolution.Permutation
            solution.TotalTransport = bestNeighborhoodSolution.TotalTransport # 这里的赋值是为了进入下一轮while循环，局部搜索bestNeighborhoodSol的邻域

        else:
            print(f'Reached local optimum of {self.Type} neighborhood. Stop local search. ')
            hasSolutionImproved = False



class SwapMove:
    def __init__(self, initialPermutation, indexA, indexB):
        self.Permutation = list(initialPermutation)
        self.IndexA = indexA
        self.IndexB = indexB

        self.Permutation[indexA] = initialPermutation[indexB]
        self.Permutation[indexB] = initialPermutation[indexA]


class SwapNeighborhood(BaseNeighborhood):
    def __init__(self, inputData, initialPermutation, evaluationLogic, solutionPool):
        super().__init__(inputData, initialPermutation, evaluationLogic, solutionPool)

        self.Type = 'Swap'

    def DiscoverMoves(self):
        for i in range(len(self.Permutation)):
            for j in range(len(self.Permutation)):
                if i < j:
                    swapMove = SwapMove(self.Permutation, i, j)
                    self.Moves.append(swapMove)


class TwoEdgeExchangeMove:
    def __init__(self, initialPermutation, indexA, indexB):
        self.Permutation = []

        self.Permutation.extend(initialPermutation[: indexA])
        self.Permutation.extend(reversed(initialPermutation[indexA: indexB]))
        self.Permutation.extend(initialPermutation[indexB: ])


# 这个类用于构建由2 opt算法得出的邻域，也就是往self.Moves里面添加TwoEdgeExchangeMove的对象
class TwoEdgeExchangeNeighborhood(BaseNeighborhood):
    def __init__(self, inputData, initialPermutation, evaluationLogic, solutionPool):
        super().__init__(inputData, initialPermutation, evaluationLogic, solutionPool)

        self.Type = 'TwoEdgeExchange'

    def DiscoverMoves(self):
        for i in range(len(self.Permutation)):
            for j in range(len(self.Permutation)):
                if j < i + 1:
                    continue
                twoEdgeMove = TwoEdgeExchangeMove(self.Permutation, i, j)
                self.Moves.append(twoEdgeMove)
        



In [36]:
class ImprovementAlgorithm:
    def __init__(self, inputData, neighborhoodTypes = ['Swap']):
        self.InputData = inputData

        self.EvaluationLogic = {}
        self.SolutionPool = {}
        self.RNG = {}

        self.NeighborhoodTypes = neighborhoodTypes
        self.Neighborhoods = {}

    def Initialize(self, evaluationLogic, solutionPool, rng = None):
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool
        self.RNG = rng


    def CreateNeighborhood(self, neighborhoodType, bestCurrentSolution):
        if neighborhoodType == 'Swap':
            return SwapNeighborhood(self.InputData, bestCurrentSolution.Permutation, self.EvaluationLogic, self.SolutionPool)
        elif neighborhoodType == 'TwoEdgeExchange':
            return TwoEdgeExchangeNeighborhood(self.InputData, bestCurrentSolution.Permutation, self.EvaluationLogic, self.SolutionPool)

        else:
            raise Exception(f'Neighborhood type {neighborhoodType} not defined. ')

    def InitializeNeighborhoods(self, solution):
        for neighborhoodType in self.NeighborhoodTypes:
            neighborhood = self.CreateNeighborhood(neighborhoodType, solution)
            self.Neighborhoods[neighborhoodType] = neighborhood
            









In [42]:
swapNeighbor = SwapNeighborhood(data, bestInitialSolution.Permutation, EvaluationLogic(False), SolutionPool())
print(swapNeighbor.Moves)

[]
